In [2]:
import csv
import numpy as np
import pandas as pd

In [3]:
def read_csv(file, read_empty_lines=True, formats={}, encoding='utf-8', delimiter=';', quotechar='"', verbose = True):
    with open (file=file, mode='r', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter, quotechar=quotechar)
        header = next(csv_reader)
        for i, column in enumerate(header):
            header[i] = column.strip().replace('\ufeff', '')
        n_columns = len(header)
        n_skipped_rows = 0
        skipped_row = []
        rows = []
        is_individual_format = False
        all_format = 0
        if 'all' in formats.keys():
            if formats['all'] == 'str':
                is_individual_format = False
                all_format = 1
        for i, row in enumerate(csv_reader):
            #print(', '.join(row))
            assert len(row) == n_columns, 'Počet sloupců je divný'
            for j, column in enumerate(header):
                if not is_individual_format and all_format == 0:
                    try:
                        cislo = int(row[j].strip().replace(' ', '').replace(',', '.'))
                    except Exception as e:
                        cislo = row[j].strip()
                        try:
                            cislo = float(row[j].strip().replace(' ', '').replace(',', '.'))
                        except Exception as e:
                            cislo = row[j].strip()
                    if str(cislo) != row[j].strip().replace(' ', '').replace(',', '.'):
                        cislo = row[j].strip()
                    #print(str(cislo), row[j].strip().replace(' ', '').replace(',', '.'))
                else:
                    cislo = row[j].strip()
                if j == 0:    #radky
                    if isinstance(cislo, str) and cislo == '' and not read_empty_lines:
                        skipped_row = ['']
                    else:
                        skipped_row = []
                        rows.append([cislo])
                else:    #cisla
                    if len(skipped_row) == 0:
                        rows[-1].append(cislo)
                    else:
                        if cislo == '':
                            skipped_row.append('')
                            if j == n_columns - 1: #last empty collumn in skipped row
                                n_skipped_rows += 1
                        else:
                            #not an empty row
                            rows.append(skipped_row)
                            skipped_row = []
                            rows[-1].append(cislo)
    assert csv_reader.line_num - 1 - n_skipped_rows == len(rows), 'Počet řádků je divný'
    if verbose:
        print(f'Skipped {n_skipped_rows} empty rows!')
    return rows, header

In [4]:
file, delimiter = '_ISCO_data/isco.csv', '\t'

In [5]:
data, columns = read_csv(file=file, delimiter=delimiter)

Skipped 0 empty rows!


In [6]:
columns

['hlavni_trida', 'trida', 'skupina', 'podskupina', 'kategorie', 'nazev']

In [7]:
data

[[0, '', '', '', '', 'Zaměstnanci v ozbrojených silách'],
 [0, '01', '', '', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '', 'Generálové a důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '01101', 'Generálové v ozbrojených silách'],
 [0, '01', '011', '0110', '01102', 'Vyšší důstojníci v ozbrojených silách'],
 [0, '01', '011', '0110', '01103', 'Nižší důstojníci v ozbrojených silách'],
 [0, '02', '', '', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '0210', '', 'Poddůstojníci v ozbrojených silách'],
 [0, '02', '021', '0210', '02100', 'Poddůstojníci v ozbrojených silách'],
 [0,
  '03',
  '',
  '',
  '',
  'Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)'],
 [0,
  '03',
  '031',
  '',
  '',
  'Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůs

In [8]:
df = pd.DataFrame(data=data, columns=columns)

In [9]:
df

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev
0,0,,,,,Zaměstnanci v ozbrojených silách
1,0,01,,,,Generálové a důstojníci v ozbrojených silách
2,0,01,011,,,Generálové a důstojníci v ozbrojených silách
3,0,01,011,0110,,Generálové a důstojníci v ozbrojených silách
4,0,01,011,0110,01101,Generálové v ozbrojených silách
...,...,...,...,...,...,...
1985,9,96,962,9629,96291,Hlídači parkovišť a tržišť
1986,9,96,962,9629,96292,Uvaděči
1987,9,96,962,9629,96293,Šatnáři
1988,9,96,962,9629,96294,Toaletáři


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hlavni_trida  1990 non-null   int64 
 1   trida         1990 non-null   object
 2   skupina       1990 non-null   object
 3   podskupina    1990 non-null   object
 4   kategorie     1990 non-null   object
 5   nazev         1990 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.4+ KB


In [11]:
df[df['trida'] == '']

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev
0,0,,,,,Zaměstnanci v ozbrojených silách
18,1,,,,,Zákonodárci a řídící pracovníci
196,2,,,,,Specialisté
668,3,,,,,Techničtí a odborní pracovníci
1078,4,,,,,Úředníci
1187,5,,,,,Pracovníci ve službách a prodeji
1363,6,,,,,"Kvalifikovaní pracovníci v zemědělství, lesnic..."
1424,7,,,,,Řemeslníci a opraváři
1680,8,,,,,"Obsluha strojů a zařízení, montéři"
1877,9,,,,,Pomocní a nekvalifikovaní pracovníci


In [12]:
from collections import namedtuple
isco = namedtuple('isco', ['hlavni_trida', 'nazev'])
iscos = []
for i in df[df['trida'] == ''].index:
    #print(type(i[0]), type(i[1]))
    #print(i[0], i[1].name)
    iscos.append(isco(df.iloc[i].hlavni_trida, df.iloc[i].nazev))

In [13]:
iscos

[isco(hlavni_trida=np.int64(0), nazev='Zaměstnanci v ozbrojených silách'),
 isco(hlavni_trida=np.int64(1), nazev='Zákonodárci a řídící pracovníci'),
 isco(hlavni_trida=np.int64(2), nazev='Specialisté'),
 isco(hlavni_trida=np.int64(3), nazev='Techničtí a odborní pracovníci'),
 isco(hlavni_trida=np.int64(4), nazev='Úředníci'),
 isco(hlavni_trida=np.int64(5), nazev='Pracovníci ve službách a prodeji'),
 isco(hlavni_trida=np.int64(6), nazev='Kvalifikovaní pracovníci v zemědělství, lesnictví a rybářství'),
 isco(hlavni_trida=np.int64(7), nazev='Řemeslníci a opraváři'),
 isco(hlavni_trida=np.int64(8), nazev='Obsluha strojů a zařízení, montéři'),
 isco(hlavni_trida=np.int64(9), nazev='Pomocní a nekvalifikovaní pracovníci')]

In [14]:
df['podskupina1'] = df['kategorie']
df.loc[df['podskupina1'] == '', 'podskupina1'] = df[df['podskupina1'] == '']['podskupina']
df['nazev1'] = df[['podskupina1', 'nazev']].astype(str).agg(' '.join, axis=1)

In [15]:
df

,hlavni_trida,trida,skupina,podskupina,kategorie,nazev,podskupina1,nazev1
0,0,,,,,Zaměstnanci v ozbrojených silách,,Zaměstnanci v ozbrojených silách
1,0,01,,,,Generálové a důstojníci v ozbrojených silách,,Generálové a důstojníci v ozbrojených silách
2,0,01,011,,,Generálové a důstojníci v ozbrojených silách,,Generálové a důstojníci v ozbrojených silách
3,0,01,011,0110,,Generálové a důstojníci v ozbrojených silách,0110,0110 Generálové a důstojníci v ozbrojených silách
4,0,01,011,0110,01101,Generálové v ozbrojených silách,01101,01101 Generálové v ozbrojených silách
...,...,...,...,...,...,...,...,...
1985,9,96,962,9629,96291,Hlídači parkovišť a tržišť,96291,96291 Hlídači parkovišť a tržišť
1986,9,96,962,9629,96292,Uvaděči,96292,96292 Uvaděči
1987,9,96,962,9629,96293,Šatnáři,96293,96293 Šatnáři
1988,9,96,962,9629,96294,Toaletáři,96294,96294 Toaletáři


In [19]:
iscos_dict = {}
for isco in iscos:
    #print(nace.nazev)
    iscos_dict[str(isco.hlavni_trida) + ' ' + isco.nazev]=df[(df['hlavni_trida']==isco.hlavni_trida) & ((df['podskupina'] != '') | (df['kategorie'] != ''))].nazev1.unique()

In [20]:
iscos_dict

{'0 Zaměstnanci v ozbrojených silách': array(['0110 Generálové a důstojníci v ozbrojených silách',
        '01101 Generálové v ozbrojených silách',
        '01102 Vyšší důstojníci v ozbrojených silách',
        '01103 Nižší důstojníci v ozbrojených silách',
        '0210 Poddůstojníci v ozbrojených silách',
        '02100 Poddůstojníci v ozbrojených silách',
        '0310 Zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)',
        '03101 Praporčíci v ozbrojených silách',
        '03102 Mužstvo v ozbrojených silách',
        '03103 Čekatelé v ozbrojených silách',
        '03109 Ostatní zaměstnanci v ozbrojených silách (kromě generálů, důstojníků a poddůstojníků)'],
       dtype=object),
 '1 Zákonodárci a řídící pracovníci': array(['1111 Představitelé zákonodárné a výkonné moci',
        '11110 Představitelé zákonodárné a výkonné moci',
        '1112 Nejvyšší státní úředníci',
        '11121 Velvyslanci, konzulové a příbuzní pracovníci',
        '11122 Nejvyšš

In [22]:
iscos_dict['9 Pomocní a nekvalifikovaní pracovníci']

array(['9111 Uklízeči a pomocníci v domácnostech (kromě hospodyní)',
       '91110 Uklízeči a pomocníci v domácnostech (kromě hospodyní)',
       '9112 Uklízeči a pomocníci v hotelích, administrativních, průmyslových a jiných objektech',
       '91121 Uklízeči a pomocníci v administrativních objektech',
       '91122 Uklízeči a pomocníci ve zdravotnických a sociálních zařízeních',
       '91123 Uklízeči a pomocníci v ubytovacích a vzdělávacích zařízeních',
       '91124 Uklízeči ve stravovacích zařízeních, potravinářských a farmaceutických výrobních prostorech',
       '91125 Uklízeči veřejných dopravních prostředků',
       '91126 Uklízeči výrobních prostor (kromě potravinářské a farmaceutické výroby) a skladů',
       '91127 Uklízeči prodejních prostor',
       '91128 Uklízeči v provozovnách osobních služeb',
       '91129 Ostatní uklízeči a pomocníci',
       '9121 Pracovníci pro ruční praní a žehlení',
       '91210 Pracovníci pro ruční praní a žehlení',
       '9122 Pracovníci pro

In [26]:
for key in iscos_dict.keys():
    print(key)

0 Zaměstnanci v ozbrojených silách
1 Zákonodárci a řídící pracovníci
2 Specialisté
3 Techničtí a odborní pracovníci
4 Úředníci
5 Pracovníci ve službách a prodeji
6 Kvalifikovaní pracovníci v zemědělství, lesnictví a rybářství
7 Řemeslníci a opraváři
8 Obsluha strojů a zařízení, montéři
9 Pomocní a nekvalifikovaní pracovníci
